In [11]:
import anthropic
import json
import re
import os
import evaluate
import polars as pl
from rich.console import Console
from dotenv import load_dotenv
from jinja2 import Environment, FileSystemLoader

## Setup

In [9]:
load_dotenv("../.env")

console = Console(style="bold green")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=ANTHROPIC_API_KEY
)

## Test Tool Avaialability

In [10]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    messages=[{
        "role": "user",
        "content": "What tools do you have available?"
    }],
    mcp_servers=[
        {
            "type": "url",
            "url": "https://mcp.deepsense.ai/icd10_codes/mcp",
            "name": "icd10-codes"
        }
    ],
    tools=[
        {
            "type": "mcp_toolset",
            "mcp_server_name": "icd10-codes"
        }
    ],
    betas=["mcp-client-2025-11-20"]
)

In [4]:
console.print(response.content[0].text)

I have access to tools for working with ICD-10 medical coding systems. Here are the available functions:

## ICD-10 Code Tools

1. **Search Codes** (`icd10-codes_search_codes`)
   - Search for ICD-10-CM diagnosis codes or ICD-10-PCS procedure codes
   - Search by code (exact or prefix match) or by description (text search)
   - Filter for HIPAA-valid codes only
   - Limit results as needed

2. **Lookup Code** (`icd10-codes_lookup_code`)
   - Get full details for a specific ICD-10 code
   - Returns descriptions and HIPAA transaction validity status
   - Works for both diagnosis (ICD-10-CM) and procedure (ICD-10-PCS) codes

3. **Validate Code** (`icd10-codes_validate_code`)
   - Check if an ICD-10 code exists and is valid
   - Verify if code is valid for HIPAA-covered transactions
   - Distinguishes between valid codes and category headers

4. **Get Hierarchy** (`icd10-codes_get_hierarchy`)
   - Get the full hierarchy of codes under a category
   - Shows category header and all child codes
   - Useful for exploring related codes within a disease category

5. **Get by Category** (`icd10-codes_get_by_category`)
   - Get diagnosis codes by ICD-10-CM chapter or category
   - Use chapter (single letter) for broad categories
   - Use category (3 characters) for specific disease groups

6. **Get by Body System** (`icd10-codes_get_by_body_system`)
   - Get ICD-10-PCS procedure codes for a specific body system
   - Can filter by section code
   - Useful for finding all procedures related to an anatomical system

These tools allow me to help you search, validate, and explore ICD-10 medical codes for both diagnoses and 
procedures. How can I assist you with ICD-10 codes today?

## Load the MDACE dataset

In [7]:
df_mdace = pl.read_parquet("../data/MDACE/parquet-splits/Inpatient-ICD-10-with-splits.parquet")
console.print(df_mdace.schema)
console.print(f"Number of rows: {df_mdace.height}")
console.print(df_mdace.head(3))
console.print(df_mdace[1].select("code").item())
console.print(df_mdace[1].select("covered_text").item())
console.print(df_mdace[1].select("text").item()[0:500])

clinical_note = df_mdace[1].select("text").item()
clinical_phrase = df_mdace[1].select("covered_text").item()


Schema({'hadm_id': Int64, 'comment': String, 'note_id': Int64, 'category': String, 'description': String, 'text': 
String, 'begin': Int64, 'end': Int64, 'covered_text': String, 'code': String, 'code_system': String, 
'code_description': String, 'type': String, 'split': String})

Number of rows: 3936

shape: (3, 14)
┌─────────┬─────────┬─────────┬───────────┬───┬─────────────┬──────────────────────┬───────┬───────┐
│ hadm_id ┆ comment ┆ note_id ┆ category  ┆ … ┆ code_system ┆ code_description     ┆ type  ┆ split │
│ ---     ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---         ┆ ---                  ┆ ---   ┆ ---   │
│ i64     ┆ str     ┆ i64     ┆ str       ┆   ┆ str         ┆ str                  ┆ str   ┆ str   │
╞═════════╪═════════╪═════════╪═══════════╪═══╪═════════════╪══════════════════════╪═══════╪═══════╡
│ 130520  ┆         ┆ 1250764 ┆ Radiology ┆ … ┆ ICD-10-PCS  ┆ Fluoroscopy of Bile  ┆ Human ┆ train │
│         ┆         ┆         ┆           ┆   ┆             ┆ Ducts usin…          ┆       ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ ICD-10-CM   ┆ Obstruction of bile  ┆ Human ┆ train │
│         ┆         ┆         ┆ summary   ┆   ┆             ┆ duct                 ┆       ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ ICD-10-CM   ┆ Obstruction of bile  ┆ Human ┆ train │
│         ┆         ┆         ┆ summary   ┆   ┆             ┆ duct                 ┆       ┆       │
└─────────┴─────────┴─────────┴───────────┴───┴─────────────┴──────────────────────┴───────┴───────┘

K83.1

Biliary obstruction

Admission Date:  [**2194-8-14**]              Discharge Date:   [**2194-8-25**]

Date of Birth:  [**2150-10-24**]             Sex:   F

Service: SURGERY

Allergies:
No Known Allergies / Adverse Drug Reactions

Attending:[**First Name3 (LF) 473**]
Chief Complaint:
1. Biliary obstruction.
2. Biliary stricture secondary to chronic pancreatitis.
3. Status post fulminant necrotizing pancreatitis.
4. Status post intraabdominal sepsis.

Major Surgical or Invasive Procedure:
1. Extensive lysis of adhesi

## Example Usage

In [21]:
# Load the Jinja2 template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('prompt_template.md.jinja2')

prompt_template = template.render(
    clinical_note=clinical_note,
    clinical_phrase=clinical_phrase
)

console.print(prompt_template, style="black on white")

You will be acting as a medical coding specialist. Your task is to identify the most specific billable ICD-10 code 
for a clinical phrase based on the context provided in a clinical note.

Here is the full clinical note for context:

<clinical_note>
Admission Date:  [**2194-8-14**]              Discharge Date:   [**2194-8-25**]

Date of Birth:  [**2150-10-24**]             Sex:   F

Service: SURGERY

Allergies:
No Known Allergies / Adverse Drug Reactions

Attending:[**First Name3 (LF) 473**]
Chief Complaint:
1. Biliary obstruction.
2. Biliary stricture secondary to chronic pancreatitis.
3. Status post fulminant necrotizing pancreatitis.
4. Status post intraabdominal sepsis.

Major Surgical or Invasive Procedure:
1. Extensive lysis of adhesions.
2. Open cholecystectomy with common bile duct exploration.
3. Partial wedge hepatectomy.
4. Choledochoduodenostomy biliary bypass.


History of Present Illness:
43F with history of severe hemorrhagic pancreatitis ([**8-15**])
complicated by pancreatic necrosis & retroperitoneal abcess
formation & ampullary stricture s/p necrosectomy and abcess
drainage [**10/2192**] with PTBD drain placement. She has had multiple
exchanges of this drain, the last on [**2194-8-7**]. She presented
for the following scheduled operation:
1. Extensive lysis of adhesions.
2. Open cholecystectomy with common bile duct exploration.
3. Partial wedge hepatectomy.
4. Choledochoduodenostomy biliary bypass.


Past Medical History:
PMH: necrotizing pancreatitis [**8-/2192**], HCV, HTN, depression,
chronic back pain, asthma

PSH: pancreatic necrosectomy, left and right peritoneal abscess
wide drainage ([**2192-10-23**]); bilateral RP abscess washout and
J-tube
placement ([**2192-10-25**]); tracheostomy ([**2192-11-1**]); PTC with
placement of 8-Fr internal/external biliary drain ([**2192-12-24**])
for ampullary stenosis with multiple subsequent dilations and
exchanges of PTC drain, most recently [**2192-4-14**] where a 12 Fr
int/ext drain was placed


Social History:
SH: 2 children.  Lives in [**Location 3610**].  Does not currently smoke
and
quit drinking alcohol since her episode of severe pancreatitis
in
[**2192**].


Family History:
FH: liver disease and bone cancer, no known pancreatic issues


Physical Exam:
Upon Discharge:
All vitals stable and within normal limits, afebrile
Gen - AAOx3, in no apparent distress
CV - RRR +S1/S2 no murmurs/rubs/gallops
Resp - CTAB no wheezes/crackles/rhonchi
Abd - soft, mildly tender to palpation appropriately near
incision, non-distended, +BS, no rebound/rigidity/guarding, no
palpable masses
Inc - clean/dry/intact, with no erythema/induration/drainage
Ext - no edema/clubbing/cyanosis

Pertinent Results:
OPERATIVE PATHOLOGY ([**8-14**]):
Gallbladder, open cholecystectomy:
     - Chronic cholecystitis.
     - Cystic lymph node with reactive, florid follicular
hyperplasia and sinus histiocytosis.

DRAIN STUDY AND REMOVAL OF DRAIN ([**8-20**]):
- Patent choledochoduodenostomy tract with free flow from the
upper common bile duct into the duodenum.  The ampulla appears
to be fully obstructed.  There was no appreciable flow along
this anatomic pathway, though this is likely just higher
resistance than the bypass.  No intrahepatic strictures
identified.  Left-sided ducts were not filled during this
examination.
- Uncomplicated removal of indwelling biliary drain.  Patient
may continue to have some leaking into the bandage.  Please
change the dressing p.r.n. with a pressure-type dressing.  The
tract should close completely in several days.

RUQ ULTRASOUND ([**8-24**]):
1.  Diffuse pneumobilia, unchanged from prior.  No significant
biliary ductal dilatation.
2.  No definite fluid collection within the region of the porta
hepatis.
Examination is limited due to overlying bowel gas.  If high
clinical
suspicion, consider CT for further assessment.
3.  Unchanged splenomegaly.
4.  Mild abdominal ascites.

BILATERAL LE ULTRASOUND ([**8-24**]): No lower extremity DVT

DISCHARGE LABS:
[**2194-8-25

In [17]:
no_tool_response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],
    messages=[
        {
            "role": "user",
            "content": prompt_template}
    ],
    output_format={
        "type": "json_schema",
        "schema": {
            "type": "object",
            "properties": {
                "ICD10": {"type": "string"},
                "description": {"type": "string"},
                "justification": {"type": "string"}
            },
            "required": ["ICD10", "description", "justification"],
            "additionalProperties": False
        }
    },
)

In [18]:
res = json.loads(no_tool_response.content[0].text)
console.print("\n[bold green]Structured Output:[/bold green]")
console.print(res)

Structured Output:

{
    'ICD10': 'K83.1',
    'description': 'Obstruction of bile duct',
    'justification': "The clinical phrase 'Biliary obstruction' with context from the note indicates obstruction of
the bile duct. The note documents biliary stricture secondary to chronic pancreatitis with ampullary stricture 
requiring choledochoduodenostomy biliary bypass. The operative findings and drain study confirmed biliary 
obstruction with patent surgical bypass created. ICD-10 code K83.1 is the most specific billable code for 
obstruction of the bile duct, which encompasses the biliary obstruction documented in this case."
}

## Exmample Usage with Tools

In [22]:
# Load the Jinja2 template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('prompt_template_tools.md.jinja2')

prompt_template = template.render(
    clinical_note=clinical_note,
    clinical_phrase=clinical_phrase
)

In [23]:
# Example: Extract ICD10 code with tool usage enabled
# Claude can use the ICD10 tools to validate and lookup codes

response_with_tools = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2048,
    messages=[{
        "role": "user",
        "content": prompt_template
    }],
    mcp_servers=[
        {
            "type": "url",
            "url": "https://mcp.deepsense.ai/icd10_codes/mcp",
            "name": "icd10-codes"
        }
    ],
    tools=[
        {
            "type": "mcp_toolset",
            "mcp_server_name": "icd10-codes"
        }
    ],
    betas=["mcp-client-2025-11-20"]
)

print(response_with_tools.content[-1].text)

<answer>
<icd10_code>K83.1</icd10_code>
<description>Obstruction of bile duct</description>
<justification>K83.1 is the most appropriate and specific billable code for "biliary obstruction" in this clinical context. The patient has biliary obstruction secondary to chronic pancreatitis with ampullary stricture/stenosis, as documented in the clinical note. This obstruction is NOT due to calculus/stones (which would require K80.xx codes), but rather is a consequence of the patient's history of severe necrotizing pancreatitis leading to ampullary stenosis and biliary stricture. The code K83.1 appropriately captures obstruction of the bile duct from non-calculous causes. The validation confirms this is a valid, billable code appropriate for HIPAA-covered transactions.</justification>
</answer>


In [24]:
console.print(response_with_tools.content, style="white on black")

[
    BetaMCPToolUseBlock(
        id='mcptoolu_01BmGhPvbTb3DKU7R3umgJQ8',
        input={'query': 'biliary obstruction', 'search_by': 'description', 'code_type': 'diagnosis'},
        name='search_codes',
        server_name='icd10-codes',
        type='mcp_tool_use'
    ),
    BetaMCPToolResultBlock(
        content=[
            BetaTextBlock(
                citations=None,
                text='{"count":20,"results":[{"code":"K83.5","short_description":"Biliary 
cyst","long_description":"Biliary 
cyst","valid_for_hipaa_transactions":true,"chapter":"K","category":"K83"},{"code":"K74.4","short_description":"Seco
ndary biliary cirrhosis","long_description":"Secondary biliary 
cirrhosis","valid_for_hipaa_transactions":true,"chapter":"K","category":"K74"},{"code":"K74.3","short_description":
"Primary biliary cirrhosis","long_description":"Primary biliary 
cirrhosis","valid_for_hipaa_transactions":true,"chapter":"K","category":"K74"},{"code":"K83.8","short_description":
"Other specified diseases of biliary tract","long_description":"Other specified diseases of biliary 
tract","valid_for_hipaa_transactions":true,"chapter":"K","category":"K83"},{"code":"K74.5","short_description":"Bil
iary cirrhosis, unspecified","long_description":"Biliary cirrhosis, 
unspecified","valid_for_hipaa_transactions":true,"chapter":"K","category":"K74"},{"code":"K83.9","short_description
":"Disease of biliary tract, unspecified","long_description":"Disease of biliary tract, 
unspecified","valid_for_hipaa_transactions":true,"chapter":"K","category":"K83"},{"code":"C24.9","short_description
":"Malignant neoplasm of biliary tract, unspecified","long_description":"Malignant neoplasm of biliary tract, 
unspecified","valid_for_hipaa_transactions":true,"chapter":"C","category":"C24"},{"code":"K85.11","short_descriptio
n":"Biliary acute pancreatitis with uninfected necrosis","long_description":"Biliary acute pancreatitis with 
uninfected 
necrosis","valid_for_hipaa_transactions":true,"chapter":"K","category":"K85"},{"code":"O26.62","short_description":
"Liver and biliary tract disorders in childbirth","long_description":"Liver and biliary tract disorders in 
childbirth","valid_for_hipaa_transactions":true,"chapter":"O","category":"O26"},{"code":"O26.63","short_description
":"Liver and biliary tract disorders in the puerperium","long_description":"Liver and biliary tract disorders in 
the 
puerperium","valid_for_hipaa_transactions":true,"chapter":"O","category":"O26"},{"code":"K85.12","short_description
":"Biliary acute pancreatitis with infected necrosis","long_description":"Biliary acute pancreatitis with infected 
necrosis","valid_for_hipaa_transactions":true,"chapter":"K","category":"K85"},{"code":"K85.10","short_description":
"Biliary acute pancreatitis without necrosis or infection","long_description":"Biliary acute pancreatitis without 
necrosis or 
infection","valid_for_hipaa_transactions":true,"chapter":"K","category":"K85"},{"code":"C24.8","short_description":
"Malignant neoplasm of overlapping sites of biliary tract","long_description":"Malignant neoplasm of overlapping 
sites of biliary 
tract","valid_for_hipaa_transactions":true,"chapter":"C","category":"C24"},{"code":"O26.612","short_description":"L
iver and biliary tract disord in preg, second trimester","long_description":"Liver and biliary tract disorders in 
pregnancy, second 
trimester","valid_for_hipaa_transactions":true,"chapter":"O","category":"O26"},{"code":"O26.611","short_description
":"Liver and biliary tract disord in pregnancy, first trimester","long_description":"Liver and biliary tract 
disorders in pregnancy, first 
trimester","valid_for_hipaa_transactions":true,"chapter":"O","category":"O26"},{"code":"O26.613","short_description
":"Liver and biliary tract disord in pregnancy, third trimester","long_description":"Liver and biliary tract 
disorders in pregnancy, third 
trimester","valid_for_hipaa_transactions":true,"chapter":"O","category":"O26"},{"code":"R93.2","short_description